## Unsupervised Learning Strategy

* Download/load SP500 stocks price data
* Aggregate on monthly level and filter top 150 most liquid stocks
* Calculate monthly returns for differnt time-horizons
* Download Fama-French Factors and calculate rolling factor betas
* For each month fit a k-means clustering algorithm to group similar assets based on their features
* For each month select assets based on the cluster and form a portfolio based on efficent frontier max sharpe ratio optimization
* Visualize portfolio returns and compare to Sp500 returns

In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.regression.rolling import RollingOLS
import yfinance as yf
import pandas_datareader.data as web
import statsmodels.api as sm
import datetime as dt
import pandas_ta
import warnings
warnings.filterwarnings("ignore")

**Download SP500 stock data**

In [73]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols = list(sp500['Symbol'].unique())
symbols

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BK',
 'BBWI',
 'BAX',
 'BDX',
 'BRK-B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF-B',
 'BLDR',
 'BG',
 'BXP',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CHRW',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG'

In [74]:
end_date = '2023-09-27'
start_date = pd.to_datetime(end_date) - pd.DateOffset(years=5)

df = yf.download(tickers=symbols,
                 start=start_date, 
                 end=end_date).stack()

df

[*********************100%%**********************]  503 of 503 completed

4 Failed downloads:
['GEV', 'SOLV', 'VLTO', 'SW']: YFChartError("%ticker%: Data doesn't exist for startDate = 1538020800, endDate = 1695787200")


Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2018-09-27 A        67.763634   70.800003   70.849998   70.099998   70.580002   
           AAL      40.727211   41.500000   42.200001   41.150002   41.230000   
           AAPL     53.586349   56.237499   56.610001   55.884998   55.955002   
           ABBV     71.712486   94.139999   94.889999   93.959999   94.349998   
           ABT      65.870636   73.019997   73.180000   72.690002   73.019997   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.736298   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211014  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.131134  176.869995  178.449997  176.270004  176.580002   

Price                   Volume  
Date       Ticker               
2018-09-27 A         1581700.0  
           AAL       5654600.0  
           AAPL    120724800.0  
           ABBV      3028600.0  
           ABT       5493900.0  
...                        ...  
2023-09-26 XYL       1322400.0  
           YUM       1500600.0  
           ZBH       3610500.0  
           ZBRA       355400.0  
           ZTS       1463200.0  

[622010 rows x 6 columns]

In [75]:
df.columns = df.columns.str.lower()
df

Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2018-09-27 A        67.763634   70.800003   70.849998   70.099998   70.580002   
           AAL      40.727211   41.500000   42.200001   41.150002   41.230000   
           AAPL     53.586349   56.237499   56.610001   55.884998   55.955002   
           ABBV     71.712486   94.139999   94.889999   93.959999   94.349998   
           ABT      65.870636   73.019997   73.180000   72.690002   73.019997   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.736298   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211014  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.131134  176.869995  178.449997  176.270004  176.580002   

Price                   volume  
Date       Ticker               
2018-09-27 A         1581700.0  
           AAL       5654600.0  
           AAPL    120724800.0  
           ABBV      3028600.0  
           ABT       5493900.0  
...                        ...  
2023-09-26 XYL       1322400.0  
           YUM       1500600.0  
           ZBH       3610500.0  
           ZBRA       355400.0  
           ZTS       1463200.0  

[622010 rows x 6 columns]

### 2. Calculate features and techinal indicators for each stock

* **Garman-Klass Volatility** - *Volatility metric that includes open, close, high, and low*
* **RSI** (Relative Strength Index) - *momentum oscillator used in technical analysis to measure the speed and change of price movements*
* **Bollinger Bands** - *measure of volatility* - middle band is simple moving average - upper and lower are +- 2 SD in period
* **ATR** (Average True Range) - *measures market volatility by calculating the average range of price movement over a specified period*
* **MACD** (Moving Average Convergence Divergence) - *momentum indicator used in technical analysis to identify trends, momentum, and potential reversal points in the price of a financial asset*
* **Dollar Volume** - *measure of the total monetary value of shares traded for a particular security or in a market over a specific period*

In [76]:
# calculate the garman_klass_volitility
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*(np.log(df['adj close'])-np.log(df['open']))**2

In [77]:
# calculate the rsi
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

In [78]:
# calculate the bollinger bands
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])


In [79]:
# calculate the ATR
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'], 
                        close=stock_data['close'], 
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

In [80]:
# calculate macd
def compute_macd(close):
    macd = pandas_ta.macd(close=close, length = 20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

In [81]:
# calculate the dollar volume
df['dollar_volume'] = df['adj close']*df['volume']/1e6
df

Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2018-09-27 A        67.763634   70.800003   70.849998   70.099998   70.580002   
           AAL      40.727211   41.500000   42.200001   41.150002   41.230000   
           AAPL     53.586349   56.237499   56.610001   55.884998   55.955002   
           ABBV     71.712486   94.139999   94.889999   93.959999   94.349998   
           ABT      65.870636   73.019997   73.180000   72.690002   73.019997   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.736298   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211014  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.131134  176.869995  178.449997  176.270004  176.580002   

Price                   volume  garman_klass_vol        rsi    bb_low  \
Date       Ticker                                                       
2018-09-27 A         1581700.0         -0.000584        NaN       NaN   
           AAL       5654600.0          0.000259        NaN       NaN   
           AAPL    120724800.0         -0.000640        NaN       NaN   
           ABBV      3028600.0         -0.029026        NaN       NaN   
           ABT       5493900.0         -0.004079        NaN       NaN   
...                        ...               ...        ...       ...   
2023-09-26 XYL       1322400.0         -0.000018  26.146737  4.485761   
           YUM       1500600.0         -0.000051  36.057161  4.811707   
           ZBH       3610500.0          0.000022  31.893232  4.745884   
           ZBRA       355400.0          0.000133  29.494977  5.400991   
           ZTS       1463200.0          0.000049  42.623477  5.153746   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
Date       Ticker                                                         
2018-09-27 A            NaN       NaN       NaN       NaN     107.181739  
           AAL          NaN       NaN       NaN       NaN     230.296087  
           AAPL         NaN       NaN       NaN       NaN    6469.201325  
           ABBV         NaN       NaN       NaN       NaN     217.188436  
           ABT          NaN       NaN       NaN       NaN     361.886687  
...                     ...       ...       ...       ...            ...  
2023-09-26 XYL     4.567684  4.649607 -0.701611 -1.727458     117.344880  
           YUM     4.841672  4.871637 -0.394780 -1.104324     183.389847  
           ZBH     4.785551  4.825217 -0.779533 -0.733477     402.696470  
           ZBRA    5.539167  5.677342 -0.889014 -1.269833      79.595386  
           ZTS     5.212559  5.271371  0.061359 -0.965577     256.251875  

[622010 rows x 14 columns]

### 3. Aggregate to monthly level and filter top 150 most liquid stocks for each month

* To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency

In [82]:
last_cols = [c for c in df.columns.unique() if c not in ['dollar_volume', 'volume', 'open', 'high', 'low', 'close']]


data = (pd.concat([df.unstack('Ticker')['dollar_volume'].resample('M').mean().stack('Ticker').to_frame('dollar_volume'),
          df.unstack()[last_cols].resample('M').last().stack('Ticker')],
            axis=1)).dropna()

data

dollar_volume   adj close  garman_klass_vol        rsi  \
Date       Ticker                                                           
2018-11-30 A          190.565782   69.393745         -0.000223  63.500031   
           AAL        317.481128   39.520947          0.000717  62.285702   
           AAPL      8402.804422   42.688763         -0.000970  35.364607   
           ABBV       518.552090   72.579735         -0.017217  61.190751   
           ABT        409.388046   67.074463         -0.003534  60.384000   
...                          ...         ...               ...        ...   
2023-09-30 OTIS       153.715885   78.028641         -0.000190  33.116174   
           ABNB      1633.500725  132.279999          0.000213  44.494127   
           CEG        196.304720  107.661484          0.000080  55.245457   
           GEHC       212.197216   66.105713          0.000185  40.922291   
           KVUE       664.360302   19.814007         -0.000410  35.706313   

                     bb_low    bb_mid   bb_high       atr      macd  
Date       Ticker                                                    
2018-11-30 A       4.102268  4.180028  4.257788 -0.954636  0.399541  
           AAL     3.561579  3.626777  3.691974  1.609222  1.462636  
           AAPL    3.707130  3.833351  3.959571 -1.029003 -1.277233  
           ABBV    4.154765  4.226265  4.297765  0.374620 -0.027624  
           ABT     4.129261  4.176752  4.224242 -0.719563  0.147163  
...                     ...       ...       ...       ...       ...  
2023-09-30 OTIS    4.365997  4.411282  4.456568 -1.028320 -1.534536  
           ABNB    4.857047  4.940924  5.024801 -1.006939 -0.037854  
           CEG     4.650304  4.690476  4.730649 -0.436215  0.366876  
           GEHC    4.155071  4.212607  4.270142 -0.893478 -1.116463  
           KVUE    3.005097  3.080010  3.154924 -0.899746 -1.435622  

[29197 rows x 9 columns]

* Calculate 5-year rolling average of dollar volume for each stock before filtering

In [83]:
data['dollar_volume'] = (data['dollar_volume'].unstack('Ticker').rolling(5*12).mean().stack())
data

dollar_volume   adj close  garman_klass_vol        rsi  \
Date       Ticker                                                           
2018-11-30 A                 NaN   69.393745         -0.000223  63.500031   
           AAL               NaN   39.520947          0.000717  62.285702   
           AAPL              NaN   42.688763         -0.000970  35.364607   
           ABBV              NaN   72.579735         -0.017217  61.190751   
           ABT               NaN   67.074463         -0.003534  60.384000   
...                          ...         ...               ...        ...   
2023-09-30 OTIS              NaN   78.028641         -0.000190  33.116174   
           ABNB              NaN  132.279999          0.000213  44.494127   
           CEG               NaN  107.661484          0.000080  55.245457   
           GEHC              NaN   66.105713          0.000185  40.922291   
           KVUE              NaN   19.814007         -0.000410  35.706313   

                     bb_low    bb_mid   bb_high       atr      macd  
Date       Ticker                                                    
2018-11-30 A       4.102268  4.180028  4.257788 -0.954636  0.399541  
           AAL     3.561579  3.626777  3.691974  1.609222  1.462636  
           AAPL    3.707130  3.833351  3.959571 -1.029003 -1.277233  
           ABBV    4.154765  4.226265  4.297765  0.374620 -0.027624  
           ABT     4.129261  4.176752  4.224242 -0.719563  0.147163  
...                     ...       ...       ...       ...       ...  
2023-09-30 OTIS    4.365997  4.411282  4.456568 -1.028320 -1.534536  
           ABNB    4.857047  4.940924  5.024801 -1.006939 -0.037854  
           CEG     4.650304  4.690476  4.730649 -0.436215  0.366876  
           GEHC    4.155071  4.212607  4.270142 -0.893478 -1.116463  
           KVUE    3.005097  3.080010  3.154924 -0.899746 -1.435622  

[29197 rows x 9 columns]